In [2]:
from collections import Counter
import random
from functools import reduce
import matplotlib.pyplot as plt
import numpy as np

## calculate score each roll

In [3]:
def getScore(dice):
    ''' takes in a list of dice number and sum up all the possible scores(include single ones and fives)
    Returns the scores, the number of dice left unscored, and the dictionary stores the dice number and the count'''
    dice.sort()
    counter = Counter(dice) # gives a dictionary stores the dice number and the count
    if len(dice)==6:   # cases score all six dice
        if dice == [1,2,3,4,5,6]: # a 1-6 straight 
            return 1500, 0, counter
        elif any(counter[i]==6 for i in counter): # 6 of a kind
            return 3000, 0, counter
        elif any(dice.count(i)==4 and dice.count(j)==2 for i in range(1,7) for j in range(1,7)): # 4 of a kind and a pair
            return 1500, 0, counter
        elif dice[0]==dice[1]==dice[2] and dice[3]==dice[4]==dice[5]: # two triplets
            return 2500, 0, counter
        elif dice[0]==dice[1] and dice[2]==dice[3] and dice[4]==dice[5]: # three pairs
            return 1500, 0, counter

    # cases score some dice in combanition plus individual ones and fives
    sum = 0
    for i in counter:
        if counter[i]==5: # 5 of a kind plus some single ones and fives
            sum += 2000
            oneFiveSum, count = oneFiveScoreCount(i, counter, dice)
            sum += oneFiveSum
            count += 5
            return sum, len(dice)-count, counter
        elif counter[i]==4: # 4 of a kind plus some single ones and fives
            sum += 1000
            oneFiveSum, count = oneFiveScoreCount(i, counter, dice)
            sum += oneFiveSum
            count += 4
            return sum, len(dice)-count, counter
        elif counter[i]==3: # 3 of a kind plus some single ones and fives
            if i==1:
                sum += 1000
            else:
                sum += i*100
            oneFiveSum, count = oneFiveScoreCount(i, counter, dice)
            sum += oneFiveSum
            count += 3
            return sum, len(dice)-count, counter
        
    # no combinations, only sum up socres of ones and fives
    oneFiveSum, count = oneFiveScoreCount(-1, counter, dice)
    return oneFiveSum, len(dice)-count, counter

def oneFiveScoreCount(curr_num, counter, dice):
    '''takes in current number, counter, and dice list.
    Returns the sum score of single ones and fives and their total count'''
    score = 0
    count = 0
    if curr_num!=1:
        score += counter[1]*100
        count += counter[1]
    if curr_num!=5:
        score += counter[5]*50
        count += counter[5]
    return score, count

## Strategies

In [4]:
def hotDice(score, numDice):
    '''check if the roll is in the hotDice situation that get score with all six dice'''
    if score!=0 and numDice==0:
        return True
    else:
        return False

In [5]:
def rollDice(numDice):
    '''roll dice'''
    return [random.randint(1, 6) for x in range(numDice)]

In [6]:
def strategy1(minScore, numDiceThresh):
    '''strategy 1:
    takes in a minimum score requirement, and a number of dice threshold.
    Keep rolling dices if you haven't meet the minimum score requirement,
    or the number of dice is higher than the threshold.
    Return the total score get in this turn.'''
    score, numDice = 0, 6
    while score < minScore or numDice > numDiceThresh:
        dice = rollDice(numDice)
        rollScore, numDice, counter = getScore(dice)
        # farkle situation, get 0 score
        if rollScore==0:
            return 0
        score += rollScore
        # hotDice situation, roll again
        if hotDice(rollScore, numDice):
            HDRollScore = strategy1(minScore-score, numDiceThresh)
            # if farkle in HotDice turn, lose all the points as well
            if HDRollScore == 0:
                return 0
            else:
                score += HDRollScore
    # minimum score requirement
    if score < minScore:
        return 0
    else:
        return score

In [7]:
def strategy2(minScore, scoreThresh):
    '''strategy 2:
    takes in a minimum score requirement, a score threshold.
    Keep rolling dices if you haven't meet the minimum score requirement,
    or score is lower than the threshold.
    Return the total score get in this turn.'''
    score, numDice = 0, 6
    while score < minScore or score < scoreThresh:
        dice = rollDice(numDice)
        rollScore, numDice, counter = getScore(dice)
        # farkle situation, get 0 score
        if rollScore==0:
            return 0
        score += rollScore
        # hotDice situation, roll again
        if hotDice(rollScore, numDice):
            HDRollScore = strategy2(minScore-score, scoreThresh-score)
            # if farkle in HotDice turn, lose all the points as well
            if HDRollScore == 0:
                return 0
            else:
                score += HDRollScore
    # minimum score requirement
    if score < minScore:
        return 0
    else:
        return score

In [8]:
def strategy3(minScore, scoreThresh, numDiceThresh):
    '''strategy 3:
    takes in a minimum score requirement, a score threshold, and a number of dice threshold.
    Keep rolling dices if you haven't meet the minimum score requirement,
    or score is lower than the threshold and the number of dice is higher than the threshold.
    Return the total score get in this turn.'''
    score, numDice = 0, 6
    while score < minScore or (numDice > numDiceThresh and score < scoreThresh):
        dice = rollDice(numDice)
        rollScore, numDice, counter = getScore(dice)
        # farkle situation, get 0 score
        if rollScore==0:
            return 0
        score += rollScore
        # hotDice situation, roll again
        if hotDice(rollScore, numDice):
            HDRollScore = strategy3(minScore-score, scoreThresh-score, numDiceThresh)
            # if farkle in HotDice turn, lose all the points as well
            if HDRollScore == 0:
                return 0
            else:
                score += HDRollScore
    # minimum score requirement
    if score < minScore:
        return 0
    else:
        return score

In [9]:
def strategy4(minScore, numDiceThresh):
    '''strategy 4:
    takes in a minimum score requirement and number of dice threshold.
    Keep rolling dices if you haven't meet the minimum score requirement,
    or the number of dice is higher than the threshold.
    If you have more than four dice and have some 1s or 5s, do not take triplet 2s or 3s.
    If you have more than three dice and no combination appear, only take one 1 if you have 1
    or take one 5 if you don't have 1.
    Return the total score get in this turn.'''
    score, numDice = 0, 6
    while score < minScore or numDice > numDiceThresh:
        oriNumDice = numDice
        dice = rollDice(numDice)
        rollScore, numDice, counter = getScore(dice)
        # farkle situation, get 0 score
        if rollScore==0:
            return 0

        # hotDice situation, roll again
        if hotDice(rollScore, numDice):
            HDRollScore = strategy4(minScore-score, numDiceThresh)
            # if farkle in HotDice turn, lose all the points as well
            if HDRollScore == 0:
                return 0
            else:
                score += HDRollScore
        else:
            # do not take triplet 2 or 3 in specific situation
            if oriNumDice>4:
                if counter[2]==3 and len(counter)>2 and (counter[1]!=0 or counter[5]!=0):
                    rollScore -= 200
                    numDice += 3
                if counter[3]==3 and len(counter)>2 and (counter[1]!=0 or counter[5]!=0):
                    rollScore -= 300
                    numDice += 3
            # take only one 1 or 5 in some situation
            if oriNumDice>3 and all(counter[i]<3 for i in counter):
                if counter[1]>0:
                    rollScore -= 100*(counter[1]-1) + 50*counter[5]
                    numDice += counter[1] + counter[5] - 1
                elif counter[5]>0:
                    rollScore -= 50*(counter[5]-1)
                    numDice += counter[5]-1
        score += rollScore
    # minimum score requirement
    if score < minScore:
        return 0
    else:
        return score

In [10]:
def strategy5(minScore, scoreThresh):
    '''strategy 5:
    takes in a minimum score requirement, a score threshold.
    Keep rolling dices if you haven't meet the minimum score requirement,
    or score is lower than the threshold.
    If you have more than four dice and have some 1s or 5s, do not take triplet 2s or 3s.
    If you have more than three dice and no combination appear, only take one 1 if you have 1
    or take one 5 if you don't have 1.
    Return the total score get in this turn.'''
    score, numDice = 0, 6
    while score < minScore or score < scoreThresh:
        oriNumDice = numDice
        dice = rollDice(numDice)
        # print("dice: ", dice)
        rollScore, numDice, counter = getScore(dice)
        # farkle situation, get 0 score
        if rollScore==0:
            return 0

        # hotDice situation, roll again
        if hotDice(rollScore, numDice):
            HDRollScore = strategy5(minScore-score, scoreThresh)
            # if farkle in HotDice turn, lose all the points as well
            if HDRollScore == 0:
                return 0
            else:
                score += HDRollScore
        else:
            # do not take triplet 2 or 3 in specific situation
            if oriNumDice>4:
                if counter[2]==3 and len(counter)>2 and (counter[1]!=0 or counter[5]!=0):
                    rollScore -= 200
                    numDice += 3
                if counter[3]==3 and len(counter)>2 and (counter[1]!=0 or counter[5]!=0):
                    rollScore -= 300
                    numDice += 3
            # take only one 1 or 5 in some situation
            if oriNumDice>3 and all(counter[i]<3 for i in counter):
                if counter[1]>0:
                    rollScore -= 100*(counter[1]-1) + 50*counter[5]
                    numDice += counter[1] + counter[5] - 1
                elif counter[5]>0:
                    rollScore -= 50*(counter[5]-1)
                    numDice += counter[5]-1
        score += rollScore
    # minimum score requirement
    if score < minScore:
        return 0
    else:
        return score

In [11]:
def strategy6(minScore, scoreThresh, numDiceThresh):
    '''strategy 6:
    takes in a minimum score requirement, a score threshold, and a number of dice threshold.
    Keep rolling dices if you haven't meet the minimum score requirement,
    or score is lower than the threshold and the number of dice is higher than the threshold.
    If you have more than four dice and have some 1s or 5s, do not take triplet 2s or 3s.
    If you have more than three dice and no combination appear, only take one 1 if you have 1
    or take one 5 if you don't have 1.
    Return the total score get in this turn.'''
    score, numDice = 0, 6
    while score < minScore or (numDice > numDiceThresh and score < scoreThresh):
        oriNumDice = numDice
        dice = rollDice(numDice)
        rollScore, numDice, counter = getScore(dice)
        # farkle situation, get 0 score
        if rollScore==0:
            return 0

        # hotDice situation, roll again
        if hotDice(rollScore, numDice):
            HDRollScore = strategy6(minScore-score, scoreThresh-score, numDiceThresh)
            # if farkle in HotDice turn, lose all the points as well
            if HDRollScore == 0:
                return 0
            else:
                score += HDRollScore
        else:
            # do not take triplet 2 or 3 in specific situation
            if oriNumDice>4:
                if counter[2]==3 and len(counter)>2 and (counter[1]!=0 or counter[5]!=0):
                    rollScore -= 200
                    numDice += 3
                if counter[3]==3 and len(counter)>2 and (counter[1]!=0 or counter[5]!=0):
                    rollScore -= 300
                    numDice += 3
            # take only one 1 or 5 in some situation
            if oriNumDice>3 and all(counter[i]<3 for i in counter):
                if counter[1]>0:
                    rollScore -= 100*(counter[1]-1) + 50*counter[5]
                    numDice += counter[1] + counter[5] - 1
                elif counter[5]>0:
                    rollScore -= 50*(counter[5]-1)
                    numDice += counter[5]-1
        score += rollScore
    # minimum score requirement
    if score < minScore:
        return 0
    else:
        return score

## Strategy Comparison

### Strategy 1 comparison

In [32]:
class Player:

    def __init__(self, total_score, wins, win_prob):
        self.total_score = total_score
        self.wins = wins
        self.win_prob = win_prob

    def CI(self):
        return min(self.win_prob), max(self.win_prob)

    def reset_score(self):
        self.total_score = 0

    def reset_wins(self):
        self.wins = 0

In [43]:
numRun = 1000
minScore = 0
numPlayers = 4
players = [None] * numPlayers
for numP in range(numPlayers): players[numP] = Player(0, 0, [])
for k in range(1000):
    for player in players: player.reset_wins()
    for l in range(numRun):
        for player in players: player.reset_score()
        winner = False
        for m in range(100):
            players[0].total_score += strategy1(minScore, 3)
            players[1].total_score += strategy1(minScore, 2)
            players[2].total_score += strategy1(minScore, 2)
            players[3].total_score += strategy1(minScore, 2)
            for player in players:
                if player.total_score >= 10000:
                    player.wins += 1
                    winner = True
            if winner: break
    players[0].win_prob.append(player.wins / numRun)

plt.hist(players[0].win_prob())

4800

In [12]:
prob = np.zeros((6,6))

wins = [0,0,0,0,0,0]
numRun = 10000
minScore = 0
for k in range(10):
    for i in range(numRun):
        score = [0,0,0,0,0,0]
        for j in range(100):
            score[0] += strategy1(minScore, 1)
            score[1] += strategy1(minScore, 2)
            score[2] += strategy1(minScore, 3)
            score[3] += strategy1(minScore, 4)
            score[4] += strategy1(minScore, 5)
            score[5] += strategy1(minScore, 6)
            if any(score[i]>=10000 for i in range(len(score))):
                score[0] += strategy1(minScore, 1)
                score[1] += strategy1(minScore, 2)
                score[2] += strategy1(minScore, 3)
                score[3] += strategy1(minScore, 4)
                score[4] += strategy1(minScore, 5)
                score[5] += strategy1(minScore, 6)
                wins[score.index(max(score))] += 1
                break
        # print("score1: ", score[0], "score2: ", score[1], "score3: ", score[2],
        #     "score4: ", score[3], "score5: ", score[4], "score6: ", score[5])
    print(wins[0]/numRun, wins[1]/numRun, wins[2]/numRun, wins[3]/numRun, wins[4]/numRun, wins[5]/numRun)

0.0917 0.2733 0.2797 0.2093 0.146 0.0
0.1923 0.5411 0.5658 0.415 0.2858 0.0


KeyboardInterrupt: 

In [ ]:
prob = np.array([[], [], [], [], [], []])
numRun = 1000
minScore = 0
numPlayers = 6
for i in range(6):
    for j in range(6):
        players=[None]*numPlayers
        for numP in range(numPlayers): players[numP] = Player(0,[])
        for k in range(10):
            for l in range(numRun):
                for m in range(100):
                    players[0].score += strategy1(minScore, i)
                    players[1].score += strategy1(minScore, j)
                    players[2].score += strategy1(minScore, j)
                    players[3].score += strategy1(minScore, j)
                    players[4].score += strategy1(minScore, j)
                    players[5].score += strategy1(minScore, j)
                    if any(ply.score>=10000 for ply in players):
                        for m in range(6):
                            if score[m] != score.index(max(score)):
                                if m == i:
                                    score[m] += strategy1(minScore, i)
                                else:
                                    score[m] += strategy1(minScore, j)
                        wins[score.index(max(score))] += 1
                        break

                

            

IndentationError: expected an indented block (3701151632.py, line 9)

In [25]:
# def compare_stategies():
aa = [1,3,2,6,3,5,8,9]
min(aa)



1

In [ ]:
wins = [0,0,0,0,0,0]
numRun = 10000
minScore = 0
for k in range(10):
    for i in range(numRun):
        score = [0,0,0,0,0,0]
        for j in range(100):
            score[0] += strategy1(minScore, 2)
            score[1] += strategy2(minScore, 500)
            score[2] += strategy3(minScore, 500, 2)
            score[3] += strategy4(minScore, 2)
            score[4] += strategy5(minScore, 500)
            score[5] += strategy6(minScore, 500, 2)
            if any(score[i]>=10000 for i in range(len(score))):
                score[0] += strategy1(minScore, 2)
                score[1] += strategy2(minScore, 500)
                score[2] += strategy3(minScore, 500, 2)
                score[3] += strategy4(minScore, 2)
                score[4] += strategy5(minScore, 500)
                score[5] += strategy6(minScore, 500, 2)
                wins[score.index(max(score))] += 1
                break
        # print("score1: ", score[0], "score2: ", score[1], "score3: ", score[2],
        #     "score4: ", score[3], "score5: ", score[4], "score6: ", score[5])
    print(wins[0]/numRun, wins[1]/numRun, wins[2]/numRun, wins[3]/numRun, wins[4]/numRun, wins[5]/numRun)

score1:  8900 score2:  7650 score3:  6350 score4:  10400 score5:  6700 score6:  9850
score1:  1900 score2:  4700 score3:  5650 score4:  6950 score5:  10450 score6:  8700
score1:  11100 score2:  6150 score3:  7150 score4:  5900 score5:  5500 score6:  8750
score1:  7050 score2:  8900 score3:  6000 score4:  9100 score5:  6050 score6:  10250
score1:  10450 score2:  5400 score3:  3300 score4:  3250 score5:  4300 score6:  10900
score1:  13550 score2:  1850 score3:  4000 score4:  5700 score5:  6600 score6:  6900
score1:  10100 score2:  5100 score3:  6800 score4:  11750 score5:  4150 score6:  10400
score1:  11950 score2:  5850 score3:  3300 score4:  3850 score5:  5500 score6:  4900
score1:  4700 score2:  5500 score3:  2350 score4:  8900 score5:  4700 score6:  12400
score1:  3100 score2:  1100 score3:  3900 score4:  10750 score5:  3500 score6:  2150
score1:  10150 score2:  4500 score3:  3350 score4:  4450 score5:  2950 score6:  9750
score1:  10300 score2:  3000 score3:  5500 score4:  4250 score

In [ ]:
wins = [0,0,0,0,0,0]
numRun = 10000
minScore = 0
for i in range(numRun):
    score = [0,0,0,0,0,0]
    for j in range(100):
        score[0] += strategy2(minScore, 0)
        score[1] += strategy2(minScore, 100)
        score[2] += strategy2(minScore, 200)
        score[3] += strategy2(minScore, 300)
        score[4] += strategy2(minScore, 400)
        score[5] += strategy2(minScore, 500)
        if any(score[i]>=10000 for i in range(len(score))):
            score[0] += strategy2(minScore, 0)
            score[1] += strategy2(minScore, 100)
            score[2] += strategy2(minScore, 200)
            score[3] += strategy2(minScore, 300)
            score[4] += strategy2(minScore, 400)
            score[5] += strategy2(minScore, 500)
            wins[score.index(max(score))] += 1
            break
    # print("score1: ", score[0], "score2: ", score[1], "score3: ", score[2],
    #       "score4: ", score[3], "score5: ", score[4], "score6: ", score[5])
print(wins[0]/numRun, wins[1]/numRun, wins[2]/numRun, wins[3]/numRun, wins[4]/numRun, wins[5]/numRun)

0.1667 0.168 0.1706 0.1626 0.1654 0.1667
